In [1]:
import cv2
import time
import pandas as pd

from ultralytics import YOLO

model = YOLO("yolov8s.pt")

video_path = "Road traffic video for object recognition.mp4"
cap = cv2.VideoCapture(video_path)
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench'] 

In [2]:
import time
import numpy as np
cap=cv2.VideoCapture('Road traffic video for object recognition.mp4')
frame_width = int(cap.get(3)) 
frame_height = int(cap.get(4))
size = (frame_width, frame_height)

result = cv2.VideoWriter('filename.avi',cv2.VideoWriter_fourcc(*'MJPG'),10, size) 

down={}
up={}

down_time1 = {}
down_time2 = {}

up_time1 = {}
up_time2 = {}

counter_down=set()
counter_up=set()

text_color = (255,255,255) 

while True:    
    ret,frame = cap.read()

    results=model.track(frame, persist=True, tracker = "bytetrack.yaml")
    annotated_frame = results[0].plot()

    a=results[0].boxes.data
    a = a.detach().cpu().numpy()  
    px=pd.DataFrame(a).astype("float")
    #print(px)
    
    list=[]
             
    for index,row in px.iterrows():
        #print(row) 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        id=int(row[4])
        d=int(row[6])
         
        cx=int(x1+x2)//2
        cy=int(y1+y2)//2
        
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
        cv2.putText(frame, str(id), (x1, y1 - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color, 1)
        cv2.putText(frame,str(class_list[d]),(x1-5,y1),cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color, 1)

        upper_bound = 190
        lower_bound = 300
        offset = 7
        distance = 80
    
        ''' condition for counting '''
        
        # Right side of the road -- (DOWN direction)
        if cx > 350:
                if upper_bound < (cy + offset) and upper_bound > (cy - offset):         
                    down[id]=cy 
                    down_time1[id] = time.time()
        
                    cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                    cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
                if id in down:
                    if lower_bound < (cy + offset) and lower_bound > (cy - offset): 
                        down_time2[id] = time.time()
                        elapsed_time = down_time2[id] - down_time1[id]
                        speed = round(((distance / elapsed_time) * 3.6),2)
                        cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                        cv2.putText(frame,str(speed)+"km/hr",(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
                        counter_down.add(id)
                        
                        
                        
        # Left side of the road -- (UP direction)
        if cx < 350:
            if lower_bound < (cy + offset) and lower_bound > (cy - offset): 
                up[id]=cy 
                up_time1[id] = time.time()
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
            if id in up:
                if upper_bound < (cy + offset) and upper_bound > (cy - offset): 
                        up_time2[id] = time.time()
                        elapsed_time = up_time2[id] - up_time1[id]
                        speed = round(((distance / elapsed_time) * 3.6),2)
                        cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                        cv2.putText(frame,str(speed)+"km/hr",(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
                        counter_up.add(id)              
                    
            
     
    red_color = (0, 0, 255)  
    blue_color =(225, 0, 0)
    
    #Upper bound line
    cv2.line(frame,(350,190),(500,190),red_color,3)  
    cv2.line(frame,(100,190),(300,190),blue_color,3) 
    #Lower bound line
    cv2.line(frame,(360,300),(700,300),red_color,3)  
    cv2.line(frame,(0,300),(280,300),blue_color,3) 
    
    
    downwards = (len(counter_down))
    upwards = (len(counter_up))
    cv2.rectangle(frame,(50,5),(200,50),(0,225,0),-1)
    cv2.putText(frame,('Down - ')+ str(downwards),(60,40),cv2.FONT_HERSHEY_TRIPLEX, 0.5, red_color, 1, cv2.LINE_AA) 
    cv2.putText(frame,('Up - ')+ str(upwards),(60,25),cv2.FONT_HERSHEY_TRIPLEX, 0.5, red_color, 1, cv2.LINE_AA) 

    
    cv2.imshow("frames",frame)
    result.write(frame)
    
    #cv2.imshow("frames",annotated_frame)
    if cv2.waitKey(10) & 0xFF == ord("q"):
        break
cap.release()
result.release() 
cv2.destroyAllWindows()


0: 384x640 6 cars, 1 truck, 138.0ms
Speed: 1.0ms preprocess, 138.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 truck, 145.0ms
Speed: 2.0ms preprocess, 145.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 truck, 119.0ms
Speed: 2.0ms preprocess, 119.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 truck, 114.0ms
Speed: 1.0ms preprocess, 114.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 truck, 132.6ms
Speed: 1.0ms preprocess, 132.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 truck, 134.0ms
Speed: 1.0ms preprocess, 134.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 truck, 116.1ms
Speed: 2.2ms preprocess, 116.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 truck, 114.0ms
Speed: 1.0ms prepr


0: 384x640 9 cars, 3 trucks, 148.0ms
Speed: 2.0ms preprocess, 148.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 3 trucks, 145.0ms
Speed: 2.0ms preprocess, 145.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 3 trucks, 147.0ms
Speed: 2.0ms preprocess, 147.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 3 trucks, 146.0ms
Speed: 2.0ms preprocess, 146.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 3 trucks, 141.0ms
Speed: 2.0ms preprocess, 141.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 3 trucks, 143.3ms
Speed: 2.0ms preprocess, 143.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 3 trucks, 146.0ms
Speed: 2.0ms preprocess, 146.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 3 trucks, 146.0ms
Speed: 2.0

Speed: 3.0ms preprocess, 140.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 3 trucks, 148.0ms
Speed: 1.0ms preprocess, 148.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 3 trucks, 140.0ms
Speed: 2.0ms preprocess, 140.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 3 trucks, 141.0ms
Speed: 2.0ms preprocess, 141.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 3 trucks, 141.0ms
Speed: 2.0ms preprocess, 141.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 3 trucks, 144.0ms
Speed: 4.0ms preprocess, 144.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 3 trucks, 139.0ms
Speed: 2.0ms preprocess, 139.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 3 trucks, 140.0ms
Speed: 1.0ms preprocess, 140.0ms inference, 2.0m


0: 384x640 7 cars, 1 truck, 141.0ms
Speed: 2.0ms preprocess, 141.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 145.0ms
Speed: 1.0ms preprocess, 145.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 158.0ms
Speed: 2.0ms preprocess, 158.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 157.0ms
Speed: 2.0ms preprocess, 157.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 140.0ms
Speed: 3.0ms preprocess, 140.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 bus, 139.0ms
Speed: 2.0ms preprocess, 139.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 145.0ms
Speed: 2.0ms preprocess, 145.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 140.0ms
Speed: 3.0ms preproc